## LABORATORIUM NR. 7

Łukasz Szydlik 331446 - 05.05.2025 - Lab. WMM gr. 101 Poniedziałek 16:15 

# Analiza obrazu - detekcja twarzy

Do realizacji zadań wykorzystywane są biblioteki OpenCV, dlib oraz InsightFace (wymaga dodatkowo pakietu onnxruntime), ponadto wykorzystywana jest biblioteka PIL/pillow do wyświetlania obrazów w notatniku. Biblioteki te muszą być zainstalowane w środowisku Python, odpowiedni plik <tt>requirements.txt</tt> został udostępniony razem z notatnikiem.

Do wykonania notatnika potrzebne są:
- obraz testowy: <tt>2_Demonstration_Demonstration_Or_Protest_2_58.jpg</tt>
- model detektora Haar: <tt>haarcascade_frontalface_default.xml</tt>
- model detektora MMOD (sieć neuronowa z biblioteki dlib): <tt>mmod_human_face_detector.dat</tt>
- modele detektora twarzy (siec neuronowa) z biblioteki insightface: katalog <tt>insightface</tt>

Wszystkie potrzebne pliki powinny znajdować się w tym samym katalogu co notatnik.

> Pracując w środowisku Google Colab należy potrzebne pliki wejściowe wgrać do środowiska notatnika - można to zrobić za pomocą interfejsu środowiska w przeglądarce, lub za pomocą widgetu 'upload'. Alternatywnie można wczytywać dane z własnego dysku Google Drive, co może być czasami przydatne - wgrywanie plików do środowiska Colab może być czasochłonne (w przypadku dużej liczby plików lub dużego ich rozmiaru), ponadto jest wymagane każdorazowo po inicjalizacji środowiska (przykłady poniżej).

In [53]:
# from google.colab import files
# uploaded = files.upload()

# # alternatywnie - czytanie plików z dysku Google Drive, w tym celu należy zamontować dysk:
# from google.colab import drive
# drive.mount("/content/drive", force_remount=True)
# # i ustawić właściwą ścieżkę do danych w zmiennej, np.:
# data_dir = "/content/drive/My Drive/Colab Notebooks/...../"
# # konieczna będzie również modyfikacja dalszych fragmentów kodu, aby uwzględnić inną lokalizację plików

In [54]:
import cv2
import dlib

# # w środowisku Colab biblioteka insightface wymaga zainstalowania
# !pip install insightface
# !pip install onnxruntime
import insightface

# from google.colab.patches import cv2_imshow  # tylko w środowisku Colab
from PIL import Image
from IPython.display import display

images_path = [
    "2_Demonstration_Demonstration_Or_Protest_2_58.jpg",
    "4_Dancing_Dancing_4_489.jpg",
    "30_Surgeons_Surgeons_30_256.jpg",
    "50_Celebration_Or_Party_houseparty_50_17.jpg",
]

In [55]:
import os

def save_result(image, filename): # rgb img
    os.makedirs("results", exist_ok=True)
    cv2.imwrite(f"results/{filename}.jpg", cv2.cvtColor(image, cv2.COLOR_RGB2BGR))


## Zadania 1-2. Detekcja twarzy

In [80]:
images = []
images_gray = []
images_rgb = []

for number, image_path in zip(range(0, len(images_path)), images_path):

    img = cv2.imread(image_path)
    # przygotowanie obrazów: monochromatycznego i RGB (cv2.imread() zwraca obraz w formacie BGR - inna kolejność składowych)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    images.append(img)
    images_gray.append(img_gray)
    images_rgb.append(img_rgb)

    # cv2_imshow(img)  # tylko w środowisku Colab
    # display(Image.fromarray(img_rgb))  # display() wymaga obrazu RGB (w przeciwieństwie do cv2.imshow(), która wymaga BGR)

    save_result(img_rgb, f"img{number+1}_normal")

## Kaskada Haara

Opracowano na podstawie: https://www.pyimagesearch.com/2021/04/05/opencv-face-detection-with-haar-cascades/

In [57]:
# utworzenie i inicjalizacja detektora
haar_detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [72]:
for number, image_path in zip(range(0, len(images_path)), images_path):
    # wywołanie detektora dla określonego obrazu (img_gray)
    # wynikiem jest lista prostokątów w formacie [x, y, width, height]
    haar_results = haar_detector.detectMultiScale(images_gray[number], scaleFactor=1.05, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)
    print(len(haar_results))

    # narysowanie wyników na kopii obrazu i wyświetlenie
    img_haar = images_rgb[number].copy()
    for (x, y, w, h) in haar_results:
        cv2.rectangle(img_haar, (x, y), (x+w, y+h), (0, 255, 0), 2)
    # display(Image.fromarray(img_haar))
    save_result(img_haar, f"img{number+1}_Haar")

23
31
1
10


## Histogram zorientowanych gradientów (HOG) z maszyną wektorów nośnych (SVM)

Opracowano na podstawie: https://www.pyimagesearch.com/2021/04/19/face-detection-with-dlib-hog-and-cnn/

In [59]:
# utworzenie i inicjalizacja detektora
hog_svm_detector = dlib.get_frontal_face_detector()

In [ ]:
for number, image_path in zip(range(0, len(images_path)), images_path):

    # wywołanie detektora dla określonego obrazu (images_rgb[number])
    # wynikiem jest lista obiektów rectangle, zawierających współrzędne lewego górnego i prawego dolnego narożnika prostokąta
    hog_svm_results = hog_svm_detector(images_rgb[number], 1)
    print(len(hog_svm_results))

    # narysowanie wyników na kopii obrazu i wyświetlenie
    img_hog_svm = images_rgb[number].copy()
    for rect in hog_svm_results:
        cv2.rectangle(img_hog_svm, (rect.left(), rect.top()), (rect.right(), rect.bottom()), (0, 255, 0), 2)
    # display(Image.fromarray(img_hog_svm))
    save_result(img_hog_svm, f"img{number+1}_HOG_SVM")

21
26
0
6


## Splotowa sieć neuronowa (CNN) z biblioteki dlib

Opracowano na podstawie: https://www.pyimagesearch.com/2021/04/19/face-detection-with-dlib-hog-and-cnn/

In [61]:
# utworzenie i inicjalizacja detektora
cnn1_detector = dlib.cnn_face_detection_model_v1('mmod_human_face_detector.dat')

In [ ]:
for number, image_path in zip(range(0, len(images_path)), images_path):

    # wywołanie detektora dla określonego obrazu (images_rgb[number])
    # wynikiem jest lista obiektów mmod_rectangle, zawierających m.in. pole rect ze współrzędnymi lewego górnego i prawego dolnego narożnika prostokąta
    cnn1_results = cnn1_detector(images_rgb[number], 1)
    print(len(cnn1_results))

    # narysowanie wyników na kopii obrazu i wyświetlenie
    img_cnn1 = images_rgb[number].copy()
    for res in cnn1_results:
        rect = res.rect
        cv2.rectangle(img_cnn1, (rect.left(), rect.top()), (rect.right(), rect.bottom()), (0, 255, 0), 2)
    # display(Image.fromarray(img_cnn1))
    save_result(img_cnn1, f"img{number+1}_CNN")

27
26
0
11


## InsightFace - inna sieć neuronowa

Opracowano na podstawie: https://github.com/deepinsight/insightface/tree/master/python-package

In [63]:
# utworzenie i inicjalizacja detektora
model_name = 'buffalo_s'  # model: small (_s), medium (_m) lub large (_l)
insf_detector = insightface.app.FaceAnalysis(name=model_name, root='insightface',
                                             allowed_modules=['detection'], providers=['CPUExecutionProvider'])
insf_detector.prepare(ctx_id=0, det_size=(1024, 1024))

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface\models\buffalo_s\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
set det-size: (1024, 1024)


In [ ]:
for number, image_path in zip(range(0, len(images_path)), images_path):

    # wywołanie detektora dla określonego obrazu
    # wynikiem jest lista obiektów, zawierających m.in. pole bbox ze współrzędnymi lewego górnego i prawego dolnego narożnika prostokąta
    insf_results = insf_detector.get(images[number])
    print(len(insf_results))

    # narysowanie wyników na kopii obrazu i wyświetlenie
    img_insf = images_rgb[number].copy()
    for res in insf_results:
    # współrzędne prostokątów sa zapisane jako liczby rzeczywiste - konwersja do liczb całkowitych
        rect = res.bbox.round().astype(int)
        cv2.rectangle(img_insf, (rect[0], rect[1]), (rect[2], rect[3]), (0, 255, 0), 2)
    # display(Image.fromarray(img_insf))
    save_result(img_insf, f"img{number+1}_InsightFace")

56
38
3
11


## Funckja do wyznaczania miar jakości

In [70]:
import pandas as pd

def face_detection_summary(name_list, GT_list, TP_list, FP_list):
    """
    Creates a combined DataFrame with face detection metrics for multiple detectors.

    Parameters:
    - name_list: list of detector names
    - GT_list: list of Ground Truth counts
    - TP_list: list of True Positives
    - FP_list: list of False Positives

    Returns:
    - pandas.DataFrame with 'Metric' as rows and each detector as a separate column
    """
    all_data = {}

    for name, GT, TP, FP in zip(name_list, GT_list, TP_list, FP_list):
        FN = GT - TP
        total_detections = TP + FP

        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / GT if GT > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        miss_rate = FN / GT if GT > 0 else 0

        all_data[name] = [
            GT,
            total_detections,
            TP,
            FN,
            FP,
            round(precision, 2),
            round(recall, 2),
            round(f1, 2),
            round(miss_rate, 2),
        ]

    metrics_labels = [
        'Ground Truth (GT)',
        'Total Detections',
        'True Positives (TP)',
        'False Negatives (FN)',
        'False Positives (FP)',
        'Precision',
        'Recall',
        'F1-score',
        'Miss Rate',
    ]

    df = pd.DataFrame(all_data, index=metrics_labels)
    return df




## Zadanie 1 - Wyniki

In [74]:
detectors_1 = {
    "name": ["Haar", "HOG SVM", "CNN MMOD", "InsightFace"],
    "GT": [104, 104, 104, 104], # reference 101
    "TP": [21, 21, 27, 56],
    "FP": [2, 0, 0, 0],
}

df_combined_summary_1 = face_detection_summary(
    name_list=detectors_1["name"],
    GT_list=detectors_1["GT"],
    TP_list=detectors_1["TP"],
    FP_list=detectors_1["FP"]
)

print("2_Demonstration_Demonstration_Or_Protest_2_58")
print(df_combined_summary_1)

2_Demonstration_Demonstration_Or_Protest_2_58
                        Haar  HOG SVM  CNN MMOD  InsightFace
Ground Truth (GT)     104.00   104.00    104.00       104.00
Total Detections       23.00    21.00     27.00        56.00
True Positives (TP)    21.00    21.00     27.00        56.00
False Negatives (FN)   83.00    83.00     77.00        48.00
False Positives (FP)    2.00     0.00      0.00         0.00
Precision               0.91     1.00      1.00         1.00
Recall                  0.20     0.20      0.26         0.54
F1-score                0.33     0.34      0.41         0.70
Miss Rate               0.80     0.80      0.74         0.46


## Zadanie 1 - Porównanie wyników
1. Haar detector (OpenCV)
    - Bardzo niska czułość (recall 0.20) – wykrywa tylko niewielką część wszyskich twarzy.
    - Precision (0.91) jest wysoka, ale duża liczba FN (83) pokazuje, że wiele twarzy zostało pominiętych.
1. HOG + SVM (Dlib)
    - Bardzo podobna skuteczność do detektora Haar – identyczna liczba TP i FN, ale brak FP (Precision = 1.00).
    - Nadal bardzo niska skuteczność detekcji – Recall = 0.20.
1. CNN MMOD (Dlib)
    - Nieco lepsze pokrycie niż poprzednie (Recall = 0.26), bez fałszywych pozytywów.
    - Zmniejszenie Miss Rate do 0.74 oznacza poprawę, ale wciąż większość twarzy jest pomijana.
1. CNN InsightFace
    - Najlepsze wyniki ze wszystkich detektorów:
    - Recall = 0.54 – wykryto ponad połowę twarzy,
    - Precision = 1.00 – zero błędnych detekcji,
    - F1-score = 0.70 – zbalansowana miara skuteczności.
    - Wciąż pominięto 48 twarzy, ale to i tak znacznie lepiej niż konkurencja.

- Precision we wszystkich detektorach jest wysoka lub idealna (1.00) – oznacza to, że mało jest błędnych detekcji.
- Problemem jest recall – czyli wykrywanie wszystkich rzeczywistych twarzy.
- Żaden z modeli nie radzi sobie z wykrywaniem twarzy od tyłu i mocno pochylonych w dół.

## Zadanie 2 - Wyniki

In [75]:
# 4_Dancing_Dancing_4_489
detectors_2 = {
    "name": ["Haar", "HOG SVM", "CNN MMOD", "InsightFace"],
    "GT": [39, 39, 39, 39], # reference 39
    "TP": [26, 26, 26, 37],
    "FP": [5, 0, 0, 1],
}

df_combined_summary_2 = face_detection_summary(
    name_list=detectors_2["name"],
    GT_list=detectors_2["GT"],
    TP_list=detectors_2["TP"],
    FP_list=detectors_2["FP"]
)

# 30_Surgeons_Surgeons_30_256
detectors_3 = {
    "name": ["Haar", "HOG SVM", "CNN MMOD", "InsightFace"],
    "GT": [5, 5, 5, 5], # reference 101
    "TP": [0, 0, 0, 3],
    "FP": [1, 0, 0, 0],
}

df_combined_summary_3 = face_detection_summary(
    name_list=detectors_3["name"],
    GT_list=detectors_3["GT"],
    TP_list=detectors_3["TP"],
    FP_list=detectors_3["FP"]
)

# 50_Celebration_Or_Party_houseparty_50_17
detectors_4 = {
    "name": ["Haar", "HOG SVM", "CNN MMOD", "InsightFace"],
    "GT": [11, 11, 11, 11], # reference 11
    "TP": [6, 6, 11, 11],
    "FP": [4, 0, 0, 0],
}

df_combined_summary_4 = face_detection_summary(
    name_list=detectors_4["name"],
    GT_list=detectors_4["GT"],
    TP_list=detectors_4["TP"],
    FP_list=detectors_4["FP"]
)

print("4_Dancing_Dancing_4_489")
print(df_combined_summary_2)
print()
print("30_Surgeons_Surgeons_30_256")
print(df_combined_summary_3)
print()
print("50_Celebration_Or_Party_houseparty_50_17")
print(df_combined_summary_4)

4_Dancing_Dancing_4_489
                       Haar  HOG SVM  CNN MMOD  InsightFace
Ground Truth (GT)     39.00    39.00     39.00        39.00
Total Detections      31.00    26.00     26.00        38.00
True Positives (TP)   26.00    26.00     26.00        37.00
False Negatives (FN)  13.00    13.00     13.00         2.00
False Positives (FP)   5.00     0.00      0.00         1.00
Precision              0.84     1.00      1.00         0.97
Recall                 0.67     0.67      0.67         0.95
F1-score               0.74     0.80      0.80         0.96
Miss Rate              0.33     0.33      0.33         0.05

30_Surgeons_Surgeons_30_256
                      Haar  HOG SVM  CNN MMOD  InsightFace
Ground Truth (GT)      5.0      5.0       5.0         5.00
Total Detections       1.0      0.0       0.0         3.00
True Positives (TP)    0.0      0.0       0.0         3.00
False Negatives (FN)   5.0      5.0       5.0         2.00
False Positives (FP)   1.0      0.0       0.0       

## Zadanie 2 - Porównanie wyników

### Zdjęcie: 4_Dancing_Dancing_4_489
Ludzie na scenie + 2 małe tłumy w tle

1. InsightFace zdecydowanie przewyższa inne detektory, wykrywając prawie wszystkie twarze (37/39).

1. Klasyczne detektory (Haar, HOG SVM, CNN MMOD) pomijają 13 twarzy – najpewniej osoby w tle lub słabo widoczne.

1. HOG SVM i CNN MMOD nie popełniły żadnych fałszywych detekcji (FP=0), co tłumaczy 100% precision, ale kosztem recall.

1. InsightFace łączy bardzo wysoki recall z minimalną liczbą FP – idealne dla analizy zdjęć tłumu.

### Zdjęcie: 30_Surgeons_Surgeons_30_256
5 chirurgów z maseczkami, zasłonięte twarze

1. Klasyczne detektory kompletnie zawiodły – nie wykryto żadnej twarzy.

1. Przyczyną są zasłonięte twarze, brak widocznych ust, nosa, szczęki.

1. Tylko InsightFace poradził sobie z tym trudnym przypadkiem – wykrył 3/5 twarzy, mimo że widoczne były praktycznie tylko oczy i czoło.

1. To dowód na większą odporność InsightFace na nietypowe ujęcia i przesłony.


### Zdjęcie: 50_Celebration_Or_Party_houseparty_50_17
11 czarnoskórych osób, dobre ustawienie, ale nierówne oświetlenie

1. CNN MMOD i InsightFace spisały się perfekcyjnie – wykryły wszystkie twarze bez błędów.

1. HOG SVM wykrył tylko 6 z 11 twarzy, ale nie popełnił fałszywych detekcji.

1. Haar znowu ma najniższą skuteczność – jego model może mieć trudności z różnorodnością oświetlenia i odcieni skóry

## Zadanie 3

In [77]:
name_list = ["Haar", "HOG SVM", "CNN MMOD", "InsightFace"]
GT_total = [x + y + z + w for x, y, z, w in zip(detectors_1["GT"], detectors_2["GT"], detectors_3["GT"], detectors_4["GT"])]
TP_total = [x + y + z + w for x, y, z, w in zip(detectors_1["TP"], detectors_2["TP"], detectors_3["TP"], detectors_4["TP"])]
FP_total = [x + y + z + w for x, y, z, w in zip(detectors_1["FP"], detectors_2["FP"], detectors_3["FP"], detectors_4["FP"])]

df_combined_summary_all = face_detection_summary(name_list, GT_total, TP_total, FP_total)
print(df_combined_summary_all)

                        Haar  HOG SVM  CNN MMOD  InsightFace
Ground Truth (GT)     159.00   159.00    159.00       159.00
Total Detections       65.00    53.00     64.00       108.00
True Positives (TP)    53.00    53.00     64.00       107.00
False Negatives (FN)  106.00   106.00     95.00        52.00
False Positives (FP)   12.00     0.00      0.00         1.00
Precision               0.82     1.00      1.00         0.99
Recall                  0.33     0.33      0.40         0.67
F1-score                0.47     0.50      0.57         0.80
Miss Rate               0.67     0.67      0.60         0.33


## Zadanie 3 - Porównanie wyników

### InsightFace
1. Zdecydowanie najlepsze ogólne wyniki:

1. Najwyższy recall (0.67) — wykrywa 2× więcej twarzy niż Haar/HOG

1. Bardzo wysoka precision (0.99) — prawie wszystkie wykrycia to trafne twarze

1. F1-score = 0.80 — zbalansowana, silna skuteczność

1. Tylko 1 fałszywa detekcja na 108 prób

### CNN MMOD
1. Zero fałszywych trafień (precision = 1.00), ale recall znacznie niższy (0.40)

1. Nie popełnia błędnych wykryć, ale pomija aż 95 twarzy — zbyt ostrożny

### HOG SVM
1. Zero fałszywych trafień (precision = 1.00), ale bardzo niski recall (0.33)

1. Skuteczność (F1-score = 0.50) mocno ograniczona przez liczbę pominiętych twarzy

### Haar
Najsłabsze wyniki:

1. Najniższy F1-score (0.47) i najwyższa liczba fałszywych wykryć (12)

1. Średnia precyzja (0.82), a i tak nie znajduje 2/3 twarzy

## Zadanie 3 - Wnioski

1. InsightFace wyraźnie dominuje pod względem ogólnej skuteczności, nawet w scenach trudnych – takich jak twarze w maseczkach, ciemne karnacje, tłumy w tle czy nierówne oświetlenie.
Uzyskuje najlepszy balans między precyzją a czułością, a liczba fałszywych wykryć jest niemal zerowa.

1. Haar cascade to najstarszy i najmniej skuteczny z testowanych detektorów. Choć działa szybko, cechuje się niską czułością (Recall) i największą liczbą fałszywych detekcji (FP) spośród wszystkich metod. Może być przydatny jedynie w bardzo prostych warunkach i tam, gdzie liczy się tylko szybkość.

1. HOG + SVM wypada nieco lepiej niż Haar – nie generuje fałszywych wykryć, ale pomija większość twarzy, zwłaszcza w złożonych scenach. Jego zaletą jest stabilność i brak błędnych alarmów, ale skuteczność ogólna pozostaje niska.

1. CNN MMOD jest kompromisem – osiąga wysoką precyzję (zero fałszywych detekcji), jednak jego czułość nadal jest niższa niż w przypadku InsightFace. Nie wykrywa wszystkich twarzy, ale działa zdecydowanie lepiej niż klasyczne metody.

## Zadanie 4

In [ ]:
parameters = [0, 1, 2]


hog_svm_detector = dlib.get_frontal_face_detector()
for parameter in parameters:
    hog_svm_results = hog_svm_detector(images_rgb[0], parameter)
    print(f"hog_svm{parameter}=" + str(len(hog_svm_results)))

    img_hog_svm = images_rgb[0].copy()
    for rect in hog_svm_results:
        cv2.rectangle(img_hog_svm, (rect.left(), rect.top()), (rect.right(), rect.bottom()), (0, 255, 0), 2)
    # display(Image.fromarray(img_hog_svm))
    save_result(img_hog_svm, f"zad4_HOG_SVM_parameter_{parameter}")


cnn1_detector = dlib.cnn_face_detection_model_v1('mmod_human_face_detector.dat')
for parameter in parameters:
    cnn1_results = cnn1_detector(images_rgb[0], parameter)
    print(f"cnn{parameter}=" + str(len(cnn1_results)))

    img_cnn1 = images_rgb[0].copy()
    for res in cnn1_results:
        rect = res.rect
        cv2.rectangle(img_cnn1, (rect.left(), rect.top()), (rect.right(), rect.bottom()), (0, 255, 0), 2)
    # display(Image.fromarray(img_cnn1))
    save_result(img_cnn1, f"zad4_CNN_parameter_{parameter}")


for parameter in [0.5, 1, 2]:
    model_name = 'buffalo_s'  # model: small (_s), medium (_m) lub large (_l)
    insf_detector = insightface.app.FaceAnalysis(name=model_name, root='insightface',
                                             allowed_modules=['detection'], providers=['CPUExecutionProvider'])
    insf_detector.prepare(ctx_id=0, det_size=(int(1024*parameter), int(1024*parameter)))
    insf_results = insf_detector.get(images[0])
    print(f"insightface{parameter}=" + str(len(insf_results)))

    img_insf = images_rgb[0].copy()
    for res in insf_results:
        rect = res.bbox.round().astype(int)
        cv2.rectangle(img_insf, (rect[0], rect[1]), (rect[2], rect[3]), (0, 255, 0), 2)
    # display(Image.fromarray(img_insf))
    save_result(img_insf, f"zad4_InsightFace_parameter{parameter}")
    

hog_svm0=6
hog_svm1=21
hog_svm2=29
cnn0=4
cnn1=27
cnn2=37
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface\models\buffalo_s\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
set det-size: (512, 512)
insightface0.5=41
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface\models\buffalo_s\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
set det-size: (1024, 1024)
insightface1=56
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface\models\buffalo_s\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
set det-size: (2048, 2048)
insightface2=64


### Zadanie 4 - Wyniki

In [87]:
# Detectors Paramater: 0, 0, (512, 512)
detectors_0 = {
    "name": ["HOG SVM", "CNN MMOD", "InsightFace"],
    "GT": [104, 104, 104], # reference 101
    "TP": [6, 4, 40],
    "FP": [0, 0, 0, 1],
}

df_combined_summary_0 = face_detection_summary(
    name_list=detectors_0["name"],
    GT_list=detectors_0["GT"],
    TP_list=detectors_0["TP"],
    FP_list=detectors_0["FP"]
)

# Detectors Paramater: 1, 1, (1024, 1024)
detectors_1 = {
    "name": ["HOG SVM", "CNN MMOD", "InsightFace"],
    "GT": [104, 104, 104], # reference 101
    "TP": [21, 27, 56],
    "FP": [0, 0, 0],
}

df_combined_summary_1 = face_detection_summary(
    name_list=detectors_1["name"],
    GT_list=detectors_1["GT"],
    TP_list=detectors_1["TP"],
    FP_list=detectors_1["FP"]
)

# Detectors Paramater: 2, 2, (2048, 2048)
detectors_2 = {
    "name": ["HOG SVM", "CNN MMOD", "InsightFace"],
    "GT": [104, 104, 104], # reference 101
    "TP": [29, 37, 63],
    "FP": [0, 0, 1],
}

df_combined_summary_2 = face_detection_summary(
    name_list=detectors_2["name"],
    GT_list=detectors_2["GT"],
    TP_list=detectors_2["TP"],
    FP_list=detectors_2["FP"]
)

print("Detectors Paramater: 0, 0, (512, 512)")
print(df_combined_summary_0)
print()
print("Detectors Paramater: 1, 1, (1024, 1024)")
print(df_combined_summary_1)
print()
print("Detectors Paramater: 2, 2, (2048, 2048)")
print(df_combined_summary_2)

Detectors Paramater: 0, 0, (512, 512)
                      HOG SVM  CNN MMOD  InsightFace
Ground Truth (GT)      104.00    104.00       104.00
Total Detections         6.00      4.00        40.00
True Positives (TP)      6.00      4.00        40.00
False Negatives (FN)    98.00    100.00        64.00
False Positives (FP)     0.00      0.00         0.00
Precision                1.00      1.00         1.00
Recall                   0.06      0.04         0.38
F1-score                 0.11      0.07         0.56
Miss Rate                0.94      0.96         0.62

Detectors Paramater: 1, 1, (1024, 1024)
                      HOG SVM  CNN MMOD  InsightFace
Ground Truth (GT)      104.00    104.00       104.00
Total Detections        21.00     27.00        56.00
True Positives (TP)     21.00     27.00        56.00
False Negatives (FN)    83.00     77.00        48.00
False Positives (FP)     0.00      0.00         0.00
Precision                1.00      1.00         1.00
Recall              

## Zadanie 4 - Porównanie i wnioski

### HOG SVM:
1. Skalowanie obrazu silnie wpływa na skuteczność: recall rośnie z 0.06 do 0.28, co oznacza 5-krotny wzrost liczby wykrytych twarzy.

1. Precision pozostaje 1.00 (brak błędnych detekcji), ale pomijanych jest wiele twarzy przy niskim przeskalowaniu.

1. Detektor jest wyraźnie niewrażliwy na małe twarze, widzi je dopiero po kilkukrotnym przeskalowaniu.

### CNN MMOD:
1. Podobna zależność jak w HOG — więcej małych twarzy znajduje się przy wyższym skalowaniu.

1. Wyniki są ogólnie lepsze niż HOG (Recall 0.36 vs 0.28 przy skali 2), ale też bardziej czasochłonne.

1. Skuteczność (F1-score) poprawia się ponad 7× między skalą 0 a 2.

### InsightFace:
1. Już przy skali 0.5 (512×512) osiąga lepsze wyniki niż HOG SVM i CNN MMOD przy skali 2.

1. Najlepsze wyniki osiąga przy 2048×2048 (Recall 0.61, F1-score 0.75).

1. Mniej zachowawaczy od innych detektorów. Miał 1 fałszywą detekcję w całym eksperymencie.

1. Przy wyższym skalowaniu znajduje coraz to bardziej rozmyte twarze. Jednakże rozmyte obiekty może również kwalifikwoać jako twarze.